In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [ ]:
import pandas as pd
import numpy as np
import json
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, RocCurveDisplay
from torch.utils.data import Dataset, DataLoader
import openai

In [ ]:
openai.api_key = '"

# **Data Wrangling**: Encounter, Allergies and Medications tables

In [ ]:
df_encounter = pd.read_csv('encounters.csv')
df_encounter.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,294d0dab-907e-8fce-7a47-0c0d322a5734,2012-04-01T09:04:48Z,2012-04-01T10:02:47Z,30a6452c-4297-a1ac-977a-6a23237c7b46,f2068cee-c75c-321d-9b2c-c33535db89c9,c3d07214-c20f-3f33-ad41-0e55adf5b024,d31fccc3-1767-390d-966a-22a5156f4219,wellness,162673000,General examination of patient (procedure),136.80,1567.00,87.20,NaN,NaN
1,2ccec874-cbaa-e280-7abb-f2bc2b603961,2013-04-07T09:04:48Z,2013-04-07T09:55:49Z,30a6452c-4297-a1ac-977a-6a23237c7b46,f2068cee-c75c-321d-9b2c-c33535db89c9,c3d07214-c20f-3f33-ad41-0e55adf5b024,d31fccc3-1767-390d-966a-22a5156f4219,wellness,162673000,General examination of patient (procedure),136.80,704.20,0.00,NaN,NaN
2,953c5138-ce17-4084-3432-1ac23f184528,2015-09-28T09:04:48Z,2015-09-28T11:02:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,db106514-f254-3402-b6a4-6d210c78c7e2,2c4b7d17-0ded-3e16-b5eb-6dda1d6a81bb,d31fccc3-1767-390d-966a-22a5156f4219,emergency,50849002,Emergency room admission (procedure),146.18,1008.98,0.00,125605004.0,Fracture of bone (disorder)
3,17dd3b88-0b85-2b6f-c342-c9d6cf5315cb,2015-10-31T11:02:48Z,2015-10-31T11:17:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,f8918a95-31e8-3ac4-8d12-29ca6080ebda,b4d9fbc9-fdca-369d-bbba-019479923f08,d31fccc3-1767-390d-966a-22a5156f4219,ambulatory,185349003,Encounter for check up (procedure),85.55,85.55,3.95,359817006.0,Closed fracture of hip (disorder)
4,0b03e41b-06a6-66fa-b972-acc5a83b134a,2016-04-10T09:04:48Z,2016-04-10T10:00:45Z,30a6452c-4297-a1ac-977a-6a23237c7b46,f2068cee-c75c-321d-9b2c-c33535db89c9,c3d07214-c20f-3f33-ad41-0e55adf5b024,d31fccc3-1767-390d-966a-22a5156f4219,wellness,162673000,General examination of patient (procedure),136.80,2039.18,464.94,NaN,NaN


In [ ]:
df_encounter= df_encounter[['Id', 'PATIENT', 'ENCOUNTERCLASS', 'DESCRIPTION', 'REASONDESCRIPTION']].drop_duplicates()
df_encounter.head()

,Id,PATIENT,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION
0,294d0dab-907e-8fce-7a47-0c0d322a5734,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN
1,2ccec874-cbaa-e280-7abb-f2bc2b603961,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN
2,953c5138-ce17-4084-3432-1ac23f184528,30a6452c-4297-a1ac-977a-6a23237c7b46,emergency,Emergency room admission (procedure),Fracture of bone (disorder)
3,17dd3b88-0b85-2b6f-c342-c9d6cf5315cb,30a6452c-4297-a1ac-977a-6a23237c7b46,ambulatory,Encounter for check up (procedure),Closed fracture of hip (disorder)
4,0b03e41b-06a6-66fa-b972-acc5a83b134a,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN


In [ ]:
df_encounter.rename(columns={'Id': 'ENCOUNTER'}, inplace=True)
df_encounter.head()

,ENCOUNTER,PATIENT,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION
0,294d0dab-907e-8fce-7a47-0c0d322a5734,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN
1,2ccec874-cbaa-e280-7abb-f2bc2b603961,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN
2,953c5138-ce17-4084-3432-1ac23f184528,30a6452c-4297-a1ac-977a-6a23237c7b46,emergency,Emergency room admission (procedure),Fracture of bone (disorder)
3,17dd3b88-0b85-2b6f-c342-c9d6cf5315cb,30a6452c-4297-a1ac-977a-6a23237c7b46,ambulatory,Encounter for check up (procedure),Closed fracture of hip (disorder)
4,0b03e41b-06a6-66fa-b972-acc5a83b134a,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN


In [ ]:
df_allergies = pd.read_csv('allergies.csv')
df_allergies.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,SYSTEM,DESCRIPTION,TYPE,CATEGORY,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2
0,2014-07-26,NaN,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,419199007,SNOMED-CT,Allergy to substance (finding),allergy,environment,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-07-26,NaN,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,84489001,SNOMED-CT,Mold (organism),allergy,environment,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-07-26,NaN,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,264287008,SNOMED-CT,Animal dander (substance),allergy,environment,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD
3,2014-07-26,NaN,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,256277009,SNOMED-CT,Grass pollen (substance),allergy,environment,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-07-26,NaN,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,782576004,SNOMED-CT,Tree pollen (substance),allergy,environment,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_allergies= df_allergies[['PATIENT', 'ENCOUNTER', 'DESCRIPTION','TYPE', 'REACTION1', 'DESCRIPTION1','SEVERITY1', 'REACTION2', 'DESCRIPTION2', 'SEVERITY2']].drop_duplicates()
df_allergies.head()

,PATIENT,ENCOUNTER,DESCRIPTION,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2
0,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Allergy to substance (finding),allergy,NaN,NaN,NaN,NaN,NaN,NaN
1,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Mold (organism),allergy,NaN,NaN,NaN,NaN,NaN,NaN
2,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD
3,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Grass pollen (substance),allergy,NaN,NaN,NaN,NaN,NaN,NaN
4,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Tree pollen (substance),allergy,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_allergies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PATIENT       92 non-null     object 
 1   ENCOUNTER     92 non-null     object 
 2   DESCRIPTION   92 non-null     object 
 3   TYPE          92 non-null     object 
 4   REACTION1     44 non-null     float64
 5   DESCRIPTION1  44 non-null     object 
 6   SEVERITY1     44 non-null     object 
 7   REACTION2     30 non-null     float64
 8   DESCRIPTION2  30 non-null     object 
 9   SEVERITY2     30 non-null     object 
dtypes: float64(2), object(8)
memory usage: 7.3+ KB


In [ ]:
df_allergies.rename(columns={'DESCRIPTION': 'ALLERGIES'}, inplace=True)
df_allergies.head()

,PATIENT,ENCOUNTER,ALLERGIES,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2
0,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Allergy to substance (finding),allergy,NaN,NaN,NaN,NaN,NaN,NaN
1,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Mold (organism),allergy,NaN,NaN,NaN,NaN,NaN,NaN
2,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD
3,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Grass pollen (substance),allergy,NaN,NaN,NaN,NaN,NaN,NaN
4,4569671e-ed39-055f-8e78-422b96c9896b,f5242c34-8841-269a-50c5-c6f623575955,Tree pollen (substance),allergy,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_medication = pd.read_csv('medications.csv')
df_medication.head()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2015-09-28T11:02:48Z,2015-10-15T09:04:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,857005,Acetaminophen 325 MG / HYDROcodone Bitartrate ...,2.51,0.0,1,2.51,NaN,NaN
1,2015-09-28T11:02:48Z,2015-10-31T11:02:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,310965,Ibuprofen 200 MG Oral Tablet,365.90,0.0,1,365.90,NaN,NaN
2,2005-11-08T20:24:07Z,NaN,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,e03e23c9-4df1-3eb6-a62d-f70f02301496,d1cea2e5-1735-089f-c72f-22ad16976663,310325,ferrous sulfate 325 MG Oral Tablet,0.15,0.0,50,7.50,NaN,NaN
3,2020-10-30T11:02:48Z,2020-11-20T11:02:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,794baa15-fe5e-c061-e188-ad59022aeea5,313782,Acetaminophen 325 MG Oral Tablet,153.58,0.0,1,153.58,NaN,NaN
4,2008-08-27T00:53:03Z,2008-08-27T00:53:03Z,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,8a84efee-6fd7-f5b3-8816-9a1c60e720be,1535362,sodium fluoride 0.0272 MG/MG Oral Gel,129.94,0.0,1,129.94,66383009.0,Gingivitis (disorder)


In [ ]:
df_medication= df_medication[['PATIENT', 'ENCOUNTER', 'DESCRIPTION']].drop_duplicates()
df_medication.head()

,PATIENT,ENCOUNTER,DESCRIPTION
0,30a6452c-4297-a1ac-977a-6a23237c7b46,953c5138-ce17-4084-3432-1ac23f184528,Acetaminophen 325 MG / HYDROcodone Bitartrate ...
1,30a6452c-4297-a1ac-977a-6a23237c7b46,953c5138-ce17-4084-3432-1ac23f184528,Ibuprofen 200 MG Oral Tablet
2,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,d1cea2e5-1735-089f-c72f-22ad16976663,ferrous sulfate 325 MG Oral Tablet
3,30a6452c-4297-a1ac-977a-6a23237c7b46,794baa15-fe5e-c061-e188-ad59022aeea5,Acetaminophen 325 MG Oral Tablet
4,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,8a84efee-6fd7-f5b3-8816-9a1c60e720be,sodium fluoride 0.0272 MG/MG Oral Gel


In [ ]:
df_medication.rename(columns={'DESCRIPTION': 'MEDICATION'}, inplace=True)
df_medication.head()

,PATIENT,ENCOUNTER,MEDICATION
0,30a6452c-4297-a1ac-977a-6a23237c7b46,953c5138-ce17-4084-3432-1ac23f184528,Acetaminophen 325 MG / HYDROcodone Bitartrate ...
1,30a6452c-4297-a1ac-977a-6a23237c7b46,953c5138-ce17-4084-3432-1ac23f184528,Ibuprofen 200 MG Oral Tablet
2,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,d1cea2e5-1735-089f-c72f-22ad16976663,ferrous sulfate 325 MG Oral Tablet
3,30a6452c-4297-a1ac-977a-6a23237c7b46,794baa15-fe5e-c061-e188-ad59022aeea5,Acetaminophen 325 MG Oral Tablet
4,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,8a84efee-6fd7-f5b3-8816-9a1c60e720be,sodium fluoride 0.0272 MG/MG Oral Gel


In [ ]:
df_1 = pd.merge(df_encounter, df_allergies, on=['PATIENT', 'ENCOUNTER'], how='left')
df_1.head()

,ENCOUNTER,PATIENT,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION,ALLERGIES,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2
0,294d0dab-907e-8fce-7a47-0c0d322a5734,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2ccec874-cbaa-e280-7abb-f2bc2b603961,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,953c5138-ce17-4084-3432-1ac23f184528,30a6452c-4297-a1ac-977a-6a23237c7b46,emergency,Emergency room admission (procedure),Fracture of bone (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17dd3b88-0b85-2b6f-c342-c9d6cf5315cb,30a6452c-4297-a1ac-977a-6a23237c7b46,ambulatory,Encounter for check up (procedure),Closed fracture of hip (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0b03e41b-06a6-66fa-b972-acc5a83b134a,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_2 = pd.merge(df_1, df_medication, on=['PATIENT', 'ENCOUNTER'], how='left')
df_2.head()

,ENCOUNTER,PATIENT,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION,ALLERGIES,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2,MEDICATION
0,294d0dab-907e-8fce-7a47-0c0d322a5734,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2ccec874-cbaa-e280-7abb-f2bc2b603961,30a6452c-4297-a1ac-977a-6a23237c7b46,wellness,General examination of patient (procedure),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,953c5138-ce17-4084-3432-1ac23f184528,30a6452c-4297-a1ac-977a-6a23237c7b46,emergency,Emergency room admission (procedure),Fracture of bone (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acetaminophen 325 MG / HYDROcodone Bitartrate ...
3,953c5138-ce17-4084-3432-1ac23f184528,30a6452c-4297-a1ac-977a-6a23237c7b46,emergency,Emergency room admission (procedure),Fracture of bone (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ibuprofen 200 MG Oral Tablet
4,17dd3b88-0b85-2b6f-c342-c9d6cf5315cb,30a6452c-4297-a1ac-977a-6a23237c7b46,ambulatory,Encounter for check up (procedure),Closed fracture of hip (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_final = df_2.drop(['ENCOUNTER', 'PATIENT'], axis = 1)

In [ ]:
df_final.drop_duplicates(inplace=True)
df_final.head()

,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION,ALLERGIES,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2,MEDICATION
0,wellness,General examination of patient (procedure),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,emergency,Emergency room admission (procedure),Fracture of bone (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acetaminophen 325 MG / HYDROcodone Bitartrate ...
3,emergency,Emergency room admission (procedure),Fracture of bone (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ibuprofen 200 MG Oral Tablet
4,ambulatory,Encounter for check up (procedure),Closed fracture of hip (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ambulatory,Encounter for check up (procedure),Gingivitis (disorder),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#df_final.to_csv('df_final.csv', index=False)

In [ ]:
df_final = df_final[df_final[['REACTION1', 'REACTION2']].notnull().all(axis=1)]
df_final.head()

,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION,ALLERGIES,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2,MEDICATION
173,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD,cetirizine hydrochloride 5 MG Oral Tablet
174,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...
179,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Cow's milk (substance),allergy,247472004.0,Wheal (finding),MILD,62315008.0,Diarrhea (finding),MILD,cetirizine hydrochloride 5 MG Oral Tablet
180,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Cow's milk (substance),allergy,247472004.0,Wheal (finding),MILD,62315008.0,Diarrhea (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...
181,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Tree nut (substance),allergy,39579001.0,Anaphylaxis (disorder),SEVERE,247472004.0,Wheal (finding),MODERATE,cetirizine hydrochloride 5 MG Oral Tablet


In [ ]:
df_final['SEVERITY_1'] = df_final['SEVERITY1'].replace(['MILD', 'MODERATE', 'SEVERE'], [1, 2, 3])

<ipython-input-20-9a7a6c549fb4>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final['SEVERITY_1'] = df_final['SEVERITY1'].replace(['MILD', 'MODERATE', 'SEVERE'], [1, 2, 3])
<ipython-input-20-9a7a6c549fb4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['SEVERITY_1'] = df_final['SEVERITY1'].replace(['MILD', 'MODERATE', 'SEVERE'], [1, 2, 3])


In [ ]:
df_final['SEVERITY_2'] = df_final['SEVERITY2'].replace(['MILD', 'MODERATE', 'SEVERE'], [1, 2, 3])

<ipython-input-21-5dc68c567b68>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final['SEVERITY_2'] = df_final['SEVERITY2'].replace(['MILD', 'MODERATE', 'SEVERE'], [1, 2, 3])
<ipython-input-21-5dc68c567b68>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['SEVERITY_2'] = df_final['SEVERITY2'].replace(['MILD', 'MODERATE', 'SEVERE'], [1, 2, 3])


In [ ]:
df_final.head()

,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION,ALLERGIES,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2,MEDICATION,SEVERITY_1,SEVERITY_2
173,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD,cetirizine hydrochloride 5 MG Oral Tablet,2,1
174,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,2,1
179,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Cow's milk (substance),allergy,247472004.0,Wheal (finding),MILD,62315008.0,Diarrhea (finding),MILD,cetirizine hydrochloride 5 MG Oral Tablet,1,1
180,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Cow's milk (substance),allergy,247472004.0,Wheal (finding),MILD,62315008.0,Diarrhea (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,1,1
181,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Tree nut (substance),allergy,39579001.0,Anaphylaxis (disorder),SEVERE,247472004.0,Wheal (finding),MODERATE,cetirizine hydrochloride 5 MG Oral Tablet,3,2


# **Zero Shot ChatGPT Prompt Engineering**

In [ ]:
response = openai.ChatCompletion.create(

 model="gpt-4",
#model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": "Decide in single word if the allergy is mild or moderate or severe. Allergy and Description: Allergy is 'Tree Nut(substance)'. Description 1 is 'Anaphylaxis (disorder)'. Description 2 is 'Dyspnea(finding)'. Is the Allergy mild, moderate or severe?"},
    ]
)

print(response.choices[0].message['content'])

Severe


In [ ]:
response = openai.ChatCompletion.create(

 model="gpt-4",
#model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": "Decide in single word if the allergy is mild or moderate or severe. Allergy and Description: Allergy is 'Tree Nut(substance)'. Description 1 is 'Dyspnea (finding)'. Description 2 is 'Eruption of skin (disorder)'. Is the Allergy mild, moderate or severe?"},
    ]
)

print(response.choices[0].message['content'])

Severe


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are an expert on allergy detection."},
        {"role": "user", "content": "Decide in single word if the allergy is mild or moderate or severe. Allergy and Description: Allergy is 'Tree Nut(substance)'. Description 1 is 'Dyspnea (finding)'. Description 2 is 'Eruption of skin (disorder)'. Is the Allergy mild, moderate or severe?"},
    ]
)

print(response.choices[0].message['content'])

Severe


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are an expert on allergy detection."},
        {"role": "user", "content": "Decide in single word if the allergy is mild, moderate or severe. Allergy and Description: Allergy is 'Fish (substance)'. Description 1 is 'Dyspnea (finding)'. Description 2 is 'Finding of vomiting (finding)'. Is the Allergy mild, moderate or severe?"},
    ]
)

print(response.choices[0].message['content'])

Severe


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are an expert on allergy detection."},
        {"role": "user", "content": "Decide in single word if the allergy is mild or moderate or severe. Allergy and Description: Allergy is 'Cow's milk (substance)'. Description 1 is 'Wheal (finding)'. Description 2 is 'Diarrhea (finding)'. Is the Allergy mild, moderate or severe?"},
    ]
)

print(response.choices[0].message['content'])

Moderate


# **Chain of Thought Prompting**

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are an expert on allergy detection and medication recommendation."},
        {"role": "user", "content": "Loratadine 5 MG chewable tablet is used for allergic reaction like Rhinoconjunctivitis,  Allergic angioedema, Wheal, Dyspnea, Anaphylaxis, Cough, Abdominal pain, vomiting, skin eruption etc. Is Loratadine effective treatment for Bee Venom? Explain in one sentence"},
    ]
)

print(response.choices[0].message['content'])

Yes, Loratadine can be effective in treating symptoms of allergic reactions to bee venom, such as hives and itching, but it may not be sufficient for severe reactions like anaphylaxis where immediate medical attention is required.


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are an expert on allergy detection and medication recommendation."},
        {"role": "user", "content": "Do you think Loratadine would be an effective treatment for mold related allergies? Explain in one sentence"},
    ]
)

print(response.choices[0].message['content'])

Yes, Loratadine can be an effective treatment for mold allergies as it is an antihistamine that works by blocking the action of histamine, a substance in the body that causes allergic symptoms.


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are an expert on allergy detection and medication recommendation."},
        {"role": "user", "content": "Other medical conditions like common cold, eczema, heat rash, sleep apnea, nasal symptoms, food intolerance, insect bites, chronic hives etc are often confused with allergies. Can we take Loratadine for such medical conditions as well? Explain in one sentence"},
    ]
)

print(response.choices[0].message['content'])

Loratadine, an antihistamine, is primarily used to alleviate allergy symptoms, but may also provide relief for symptoms linked with colds, eczema, and hives, though it won't treat the underlying cause of these conditions.


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are a medical expert."},
        {"role": "user", "content": "Other medical conditions that are non allergic conditions like common cold, eczema, heat rash, sleep apnea, nasal symptoms, food intolerance, insect bites, chronic hives etc are often confused with allergies. Can Loratadine be a cure for such medical conditions? Explain in one sentence"},
    ]
)

print(response.choices[0].message['content'])

While Loratadine can help manage symptoms related to these conditions, it is not a cure as it primarily functions as an antihistamine used to alleviate allergy symptoms and not the underlying cause of these non-allergic conditions.


# **Tree of Thoughts Approach**

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are a medical expert."},
        {"role": "user", "content": "Your task is to assist with complicated treatment related problems. Acknowledge by answering 'Yes'."},
    ]
)

print(response.choices[0].message['content'])

Yes


In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are a medical expert."},
        {"role": "user", "content": "I love to eat fish, but have very bad allergic reaction to Fish. Each time I get different allergic reactions. Sometimes it is mild like whealing. But I do get moderate allergic reactions like Dyspnea to severe one like Anaphylaxis."},
        {"role": "user", "content": "Can you help me understand the reasons why I get different kinds of allergic reactions from mild to severe?"},
    ]
)

print(response.choices[0].message['content'])

Allergies occur when your immune system reacts to a foreign substance such as pollen, bee venom or a particular food such as fish in your case. The severity of an allergy from food can vary from mild symptoms to severe reactions, and they can affect various parts of the body at the same time.

The range of allergic reactions you experience might be due to several factors:

1. **Amount consumed**: The quantity of fish you consume might affect your reaction. Eating a larger amount could lead to a more severe response.

2. **Preparation of the fish**: Some people are allergic to certain proteins in fish that can be altered with cooking or processing. Depending on how the fish is prepared, you might be more or less reactive to these proteins.

3. **Individual health status**: If your immune system is compromised due to another illness or if you're feeling particularly run-down, your body might respond differently than when you're in ideal health.

4. **Repeated exposure**: Sometimes, recur

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are a medical expert."},
        {"role": "user", "content": "For each of those three kinds of allergic reaction from mild to severe due to fish consumption, could advise which medication helps for each kind?"},
    ]
)

print(response.choices[0].message['content'])

Fish allergies can cause a range of reactions, from mild to severe. Here is a general guide to how these reactions might be treated. However, keep in mind that individual treatment may vary based on specific patient needs and medical history. Always consult with a healthcare provider or allergist for appropriate treatment.

1. Mild Reaction: Minor symptoms such as isolated hives or rash may respond to over-the-counter antihistamines like diphenhydramine (Benadryl).

2. Moderate Reaction: More widespread skin symptoms or mild respiratory or gastrointestinal symptoms might necessitate stronger anti-allergy medication. Histamine H2-blockers like ranitidine (Zantac) or prescription antihistamines like desloratadine (Clarinex) might be recommended. Corticosteroids like prednisone could be used in some situations.

3. Severe Reaction (Anaphylaxis): This is a life-threatening emergency that can cause severe breathing difficulties, decreased blood pressure, and loss of consciousness. It requir

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are a medical expert."},
        {"role": "user", "content": "For each suggested medications for the ranges of allergic reaction, what are the potential challenges related to accessibility? What is the probability of cure with those recommended medications?"},
   ]
)

print(response.choices[0].message['content'])

1. Antihistamines: These are the go-to medications for mild to moderate allergic reactions causing symptoms like itching, rash, or hives. However, accessibility can be a challenge for individuals without access to pharmacies, or those living in lower-income areas where such medications might not be readily available. The probability of cure with antihistamines is high as they can provide effective relief from symptoms, but they do not address the underlying allergy – they only manage symptoms. 

2. Epinephrine (EpiPen): This is a lifesaving medication used during severe, life-threatening allergic reactions (anaphylaxis) to help relax muscles in the lungs and tighten the blood vessels. However, EpiPens can be expensive and not covered by all forms of insurance, making accessibility a challenge. Also, they must be used promptly and correctly for them to be effective. They're intended to provide temporary relief and allow time to seek emergency medical attention – they do not 'cure' the a

# **Prediction**



In [ ]:
df = df_final[df_final[['REACTION1', 'REACTION2']].notnull().all(axis=1)]
df.head()

,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION,ALLERGIES,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2,MEDICATION,SEVERITY_1,SEVERITY_2
173,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD,cetirizine hydrochloride 5 MG Oral Tablet,2,1
174,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,247472004.0,Wheal (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,2,1
179,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Cow's milk (substance),allergy,247472004.0,Wheal (finding),MILD,62315008.0,Diarrhea (finding),MILD,cetirizine hydrochloride 5 MG Oral Tablet,1,1
180,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Cow's milk (substance),allergy,247472004.0,Wheal (finding),MILD,62315008.0,Diarrhea (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,1,1
181,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Tree nut (substance),allergy,39579001.0,Anaphylaxis (disorder),SEVERE,247472004.0,Wheal (finding),MODERATE,cetirizine hydrochloride 5 MG Oral Tablet,3,2


In [ ]:
data_index = list(df.index)
train_index, test_index = train_test_split(data_index, test_size=0.2, random_state=42)

In [ ]:
print(train_index)
print(test_index)

[180, 416, 2959, 4023, 2808, 1409, 1066, 4585, 1407, 1061, 1408, 3924, 2955, 173, 3923, 4586, 182, 1064, 4027, 174, 4583, 2814, 179, 4096, 2818, 4569, 1063, 2817, 2807, 6507, 2813, 417, 1406, 3920, 4576]
[4575, 2819, 2820, 4570, 4097, 4584, 181, 1065, 896]


In [ ]:
class AllergySeverity(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        column_names = [
          ("ENCOUNTERCLASS", "The encounter class is "),
          ("ALLERGIES", ". The kind of allergy is "),
          ("DESCRIPTION1", ". The first allergic description is "),
          ("DESCRIPTION2", ". The second allergic description is ")
        ]

        x_strs = [f"{col_desc}{self.df.iloc[index][col]}" for col, col_desc in column_names]
        x_str = ''.join(x_strs)
        x_str = x_str.replace('\n', '')
        x_str = 'The allergic reaction to any thing or substance can range from mild like whealing, cutaneous hypersensitivity etc to moderate like eruption of skin, dyspnea etc to severe like Anaphylaxis.'+x_str
        x_str = x_str+'. Please use only one number like 1, 2 or 3 to decide if the allergic reaction is mild, moderate or severe when mild is 1, moderate is 2 and severe is 3.'

        return x_str

In [ ]:
allergy_dataset = AllergySeverity(df)
for i in range(len(allergy_dataset)):
    print(allergy_dataset[i])

The allergic reaction to any thing or substance can range from mild like whealing, cutaneous hypersensitivity etc to moderate like eruption of skin, dyspnea etc to severe like Anaphylaxis.The encounter class is ambulatory. The kind of allergy is Animal dander (substance). The first allergic description is Rhinoconjunctivitis (disorder). The second allergic description is Wheal (finding). Please use only one number like 1, 2 or 3 to decide if the allergic reaction is mild, moderate or severe when mild is 1, moderate is 2 and severe is 3.
The allergic reaction to any thing or substance can range from mild like whealing, cutaneous hypersensitivity etc to moderate like eruption of skin, dyspnea etc to severe like Anaphylaxis.The encounter class is ambulatory. The kind of allergy is Animal dander (substance). The first allergic description is Rhinoconjunctivitis (disorder). The second allergic description is Wheal (finding). Please use only one number like 1, 2 or 3 to decide if the allergi

In [ ]:
test = test_index
test_df = df.loc[test]
test_dataset = AllergySeverity(test_df)

In [ ]:
test

[4575, 2819, 2820, 4570, 4097, 4584, 181, 1065, 896]

In [ ]:
test_df

,ENCOUNTERCLASS,DESCRIPTION,REASONDESCRIPTION,ALLERGIES,TYPE,REACTION1,DESCRIPTION1,SEVERITY1,REACTION2,DESCRIPTION2,SEVERITY2,MEDICATION,SEVERITY_1,SEVERITY_2
4575,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Animal dander (substance),allergy,878820003.0,Rhinoconjunctivitis (disorder),MODERATE,49727002.0,Cough (finding),MODERATE,Loratadine 5 MG Chewable Tablet,2,2
2819,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Peanut (substance),allergy,402387002.0,Allergic angioedema (disorder),MODERATE,56018004.0,Wheezing (finding),MILD,cetirizine hydrochloride 5 MG Oral Tablet,2,1
2820,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Peanut (substance),allergy,402387002.0,Allergic angioedema (disorder),MODERATE,56018004.0,Wheezing (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,2,1
4570,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Latex (substance),allergy,402387002.0,Allergic angioedema (disorder),MODERATE,247472004.0,Wheal (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,2,1
4097,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Tree nut (substance),allergy,267036007.0,Dyspnea (finding),MODERATE,271807003.0,Eruption of skin (disorder),MODERATE,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,2,2
4584,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Fish (substance),allergy,267036007.0,Dyspnea (finding),MODERATE,300359004.0,Finding of vomiting (finding),MILD,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,2,1
181,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Tree nut (substance),allergy,39579001.0,Anaphylaxis (disorder),SEVERE,247472004.0,Wheal (finding),MODERATE,cetirizine hydrochloride 5 MG Oral Tablet,3,2
1065,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Fish (substance),allergy,267036007.0,Dyspnea (finding),MODERATE,271807003.0,Eruption of skin (disorder),MILD,Fexofenadine hydrochloride 30 MG Oral Tablet,2,1
896,ambulatory,Encounter for problem (procedure),Allergy to substance (finding),Aspirin,allergy,21522001.0,Abdominal pain (finding),MODERATE,402387002.0,Allergic angioedema (disorder),MODERATE,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,2,2


In [ ]:
test_dataset = AllergySeverity(test_df)

for i in range(len(test_dataset)):
    print(test_dataset[i])

The allergic reaction to any thing or substance can range from mild like whealing, cutaneous hypersensitivity etc to moderate like eruption of skin, dyspnea etc to severe like Anaphylaxis.The encounter class is ambulatory. The kind of allergy is Animal dander (substance). The first allergic description is Rhinoconjunctivitis (disorder). The second allergic description is Cough (finding). Please use only one number like 1, 2 or 3 to decide if the allergic reaction is mild, moderate or severe when mild is 1, moderate is 2 and severe is 3.
The allergic reaction to any thing or substance can range from mild like whealing, cutaneous hypersensitivity etc to moderate like eruption of skin, dyspnea etc to severe like Anaphylaxis.The encounter class is ambulatory. The kind of allergy is Peanut (substance). The first allergic description is Allergic angioedema (disorder). The second allergic description is Wheezing (finding). Please use only one number like 1, 2 or 3 to decide if the allergic re

In [ ]:
results = []
for prompt in tqdm(test_dataset):
  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
          {"role": "user", "content": prompt},
      ]
  )
  results.append(response.choices[0].message['content'])
  time.sleep(3)

100%|██████████| 9/9 [00:30<00:00,  3.35s/it]


In [ ]:
results

['1', '2', '1', '2', '2', '2', '3', '2', '2']

In [ ]:
#convert to integers using list comprehension
results_list = [int(item) for item in results]

print(results_list)

[1, 2, 1, 2, 2, 2, 3, 2, 2]


In [ ]:
test_df['SEVERITY1'] = test_df['SEVERITY1'].replace(['MILD','MODERATE', 'SEVERE'], [1, 2, 3])

<ipython-input-109-fe8e92c10817>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['SEVERITY1'] = test_df['SEVERITY1'].replace(['MILD','MODERATE', 'SEVERE'], [1, 2, 3])


In [ ]:
test_df['SEVERITY1'] = test_df['SEVERITY1'].astype(int)

In [ ]:
test_labels = list(test_df['SEVERITY1'])
test_labels

[2, 2, 2, 2, 2, 2, 3, 2, 2]

In [ ]:
test_labels = list(test_df['SEVERITY1'])
test_pred = results_list
auroc = roc_auc_score(test_labels, test_pred)
auprc = average_precision_score(test_labels, test_pred, pos_label=2)
print('\nAUROC:', auroc, '\nAUPRC', auprc)


AUROC: 1.0 
AUPRC 0.865079365079365
